# COMPROBANDO EL FLUJO SEGUIDO DEL MAIN.PY

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from services.login_service import login
from services.autorizacion_service import check_authorization
from services.control_entregas_service import manage_delivery_control
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


# Agregar al inicio del script para suprimir logs no deseados
from selenium.webdriver.remote.remote_connection import LOGGER

In [1]:
numeros_autorizacion = [
    "29766906",
    "29766636",
    "29766521",
    "29809259",
    "29809115",
    "29808833",
    "29808738"
]

In [19]:
# En la parte donde inicias el navegador, agregar opciones para suprimir logs:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--log-level=3')
options.add_argument('--disable-logging')

# Iniciar navegador con las opciones
driver = webdriver.Chrome(options=options)
# Iniciar sesi√≥n una sola vez al principio
#driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15)
login(driver, wait)

In [20]:
numero = "29766906"
tiempo_limite_minutos=2

inicio = datetime.now()
tiempo_limite = timedelta(minutes=tiempo_limite_minutos)

print(f"\nüîÅ Consultando autorizaci√≥n: {numero}\n")

try:
    # Volver a la p√°gina inicial para asegurar estado consistente
    driver.get("https://conexiones.saviasaludeps.com/savia/home.faces")
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "h3")))
    
    # Consultar autorizaci√≥n
    if not check_authorization(driver, wait, numero):
        print(f"‚ùå Fall√≥ consulta para {numero}")
        ##return False
        
    if datetime.now() - inicio > tiempo_limite:
        print(f"‚è∞ Tiempo l√≠mite excedido para {numero}")
        ##return False
        
    # Volver a cargar p√°gina antes de control de entregas
    #driver.get("https://conexiones.saviasaludeps.com/savia/home.faces")
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "h3")))

    # Gestionar control de entregas
    if not manage_delivery_control(driver, wait, numero):
        print(f"‚ùå Fall√≥ control de entregas para {numero}")
        #return False
            
    #return True

    
except Exception as e:
    print(f"‚ùå Error cr√≠tico al procesar {numero}: {str(e)}")
    #return False

except Exception as e:
    print(f"‚ùå Error al procesar autorizaci√≥n {numero}: {str(e)}")
    #return False


üîÅ Consultando autorizaci√≥n: 29766906

‚úÖ Entraste al m√≥dulo de 'Autorizaciones'
üîç Estado encontrado en la primera fila: Autorizada
‚úÖ El estado es 'Autorizada'. Se proceder√° con el clic en 'Ver'.
‚úÖ Datos de autorizaci√≥n guardados en 'data\autorizaciones\datos_autorizacion_29766906.json'
‚úÖ Di√°logo de Autorizaciones cerrado con el bot√≥n X
‚úÖ Entraste al m√≥dulo de 'Autorizaciones'
‚úÖ Se hizo clic en 'Control y registro de entregas' correctamente.
‚úÖ Datos de control de entregas guardados en 'data\control_entregas\control_entregas_29766906.json'
‚úÖ Di√°logo de control cerrado con el bot√≥n X (selector espec√≠fico)


In [11]:
try:
        wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//h3[a[text()='Autorizaciones']]"))).click().click()
        time.sleep(1)
except:
        pass

In [4]:
def process_authorization(driver, wait, numero, tiempo_limite_minutos=2):
    inicio = datetime.now()
    tiempo_limite = timedelta(minutes=tiempo_limite_minutos)
    
    print(f"\nüîÅ Consultando autorizaci√≥n: {numero}\n")
    
    try:
        # Volver a la p√°gina inicial para asegurar estado consistente
        driver.get("https://conexiones.saviasaludeps.com/savia/home.faces")
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "h3")))
        
        # Consultar autorizaci√≥n
        if not check_authorization(driver, wait, numero):
            print(f"‚ùå Fall√≥ consulta para {numero}")
            return False
            
        if datetime.now() - inicio > tiempo_limite:
            print(f"‚è∞ Tiempo l√≠mite excedido para {numero}")
            return False
            
        # Volver a cargar p√°gina antes de control de entregas
        driver.get("https://conexiones.saviasaludeps.com/savia/home.faces")
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "h3")))
        
        # Gestionar control de entregas
        if not manage_delivery_control(driver, wait, numero):
            print(f"‚ùå Fall√≥ control de entregas para {numero}")
            return False
            
        return True
        
    except Exception as e:
        print(f"‚ùå Error cr√≠tico al procesar {numero}: {str(e)}")
        return False
    
    except Exception as e:
        print(f"‚ùå Error al procesar autorizaci√≥n {numero}: {str(e)}")
        return False

In [ ]:
if __name__ == "__main__":
    driver = None
    
    try:
        # En la parte donde inicias el navegador, agregar opciones para suprimir logs:
        options = webdriver.ChromeOptions()
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        options.add_argument('--log-level=3')
        options.add_argument('--disable-logging')

        # Iniciar navegador con las opciones
        driver = webdriver.Chrome(options=options)
        # Iniciar sesi√≥n una sola vez al principio
        #driver = webdriver.Chrome()
        wait = WebDriverWait(driver, 15)
        login(driver, wait)
        
        for numero in numeros_autorizacion:
            exito = process_authorization(driver, wait, numero)
            
            if not exito:
                # Si fall√≥ o se excedi√≥ el tiempo, reiniciar el navegador
                print("üîÑ Reiniciando navegador...")
                driver.quit()
                time.sleep(2)
                
                driver = webdriver.Chrome()
                wait = WebDriverWait(driver, 15)
                login(driver, wait)
                
    finally:
        if driver:
            driver.quit()

# COMRPROBANDO EL ESTADO DE LAS AUTORIZACIONES EN EL RPA

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from services.login_service import login
from services.autorizacion_service import check_authorization
from services.control_entregas_service import manage_delivery_control
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


# Agregar al inicio del script para suprimir logs no deseados
from selenium.webdriver.remote.remote_connection import LOGGER

In [ ]:
numeros_autorizacion = [
    #"29766906",
    #"29799161"
    #"29766636",
    #"29766521",
    #"29809259",
    #"29809115",
    #"29808833",
    #"29808738"
]

In [56]:
# En la parte donde inicias el navegador, agregar opciones para suprimir logs:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--log-level=3')
options.add_argument('--disable-logging')

# Iniciar navegador con las opciones
driver = webdriver.Chrome(options=options)
# Iniciar sesi√≥n una sola vez al principio
#driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15)
login(driver, wait)

In [58]:
numero_autorizacion = [
    #"29766906",
    #"29820829", #ANULADA, 
    #"29799161" #ANULADA,
    #"29766636",
    "29766521",
    #"29809259",
    #"29809115",
    #"29808833",
    #"29808738"
]

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//h3[a[text()='Autorizaciones']]"))).click().click()
    time.sleep(1)
except:
    pass

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//span[text()='Autorizaciones']/parent::a"))).click()
    print("‚úÖ Entraste al m√≥dulo de 'Autorizaciones'")
except Exception as e:
    print("‚ùå No se pudo acceder al enlace de 'Autorizaciones':", e)
    #Return

try:
    input_busqueda = wait.until(EC.presence_of_element_located(
        (By.ID, "frmAutorizaciones:tablaRegistros:j_idt80")))
    input_busqueda.clear()
    input_busqueda.send_keys(numero_autorizacion)
except Exception as e:
    print("‚ùå No se pudo ingresar el n√∫mero de autorizaci√≥n:", e)
    #return

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//span[contains(@class, 'ui-icon-refresh')]/ancestor::button"))).click()
except Exception as e:
    print("‚ùå No se pudo hacer clic en Refrescar:", e)
    #return

time.sleep(0.5)

try:
    # Esperar a que al menos una fila con la columna "Estado" est√© visible
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//td[span[@class="ui-column-title" and text()="Estado"]]'))
    )
 
    # Obtener la primera celda con columna "Estado"
    cell = driver.find_element(By.XPATH, '(//td[span[@class="ui-column-title" and text()="Estado"]])[1]')
    full_text = cell.text.strip()
 
    # Remover el encabezado si viene incluido
    if full_text.startswith("Estado"):
        text=full_text.replace("Estado", "").strip()
    else:
        text=full_text
    print(f"Estado {text}")

    if text != "Autorizada":
            print("‚ö†Ô∏è El estado no es 'Autorizada'. Se detiene el proceso.")
            #return
    else:
        print("‚úÖ El estado es 'Autorizada'. Se proceder√° con el clic en 'Ver'.")
        #except Exception as e:
        #    print("‚ùå Error al verificar el estado de la tabla:", e)
        #return
        
 
except Exception as e:
    print(f"‚ùå No se pudo obtener el estado de la primera fila: {e}")

‚úÖ Entraste al m√≥dulo de 'Autorizaciones'
Estado Autorizada
‚úÖ El estado es 'Autorizada'. Se proceder√° con el clic en 'Ver'.


In [49]:
print(text)

Anulada


In [ ]:
# üîç 4.5 Verificar si el estado de la fila es 'Autorizada'
try:
    wait.until(EC.presence_of_element_located((By.ID, "frmAutorizaciones:tablaRegistros_data")))
    estado_xpath = "//*[@id='frmAutorizaciones:tablaRegistros_data']/tr/td[6]"
    #//*[@id="frmAutorizaciones:tablaRegistros_data"]/tr/td[6]
    estado_element = driver.find_element(By.XPATH, estado_xpath)
    estado_texto = estado_element.text.strip()
    print(f"üîç Estado encontrado en la primera fila: {estado_texto}")

    if estado_texto != "Autorizada":
        print("‚ö†Ô∏è El estado no es 'Autorizada'. Se detiene el proceso.")
        #return
    else:
        print("‚úÖ El estado es 'Autorizada'. Se proceder√° con el clic en 'Ver'.")
except Exception as e:
    print("‚ùå Error al verificar el estado de la tabla:", e)
    #return

In [ ]:
# üîç 4.5 Verificar si el estado de la fila es 'Autorizada'
try:
    wait.until(EC.presence_of_element_located((By.ID, "frmAutorizaciones:tablaRegistros_data")))
    estado_xpath = "//tbody[@id='frmAutorizaciones:tablaRegistros_data']/tr[1]/td[6]"
    estado_element = driver.find_element(By.XPATH, estado_xpath)
    estado_texto = estado_element.text.strip()
    print(f"üîç Estado encontrado en la primera fila: {estado_texto}")

    if estado_texto != "Autorizada":
        print("‚ö†Ô∏è El estado no es 'Autorizada'. Se detiene el proceso.")
        #return
    else:
        print("‚úÖ El estado es 'Autorizada'. Se proceder√° con el clic en 'Ver'.")
except Exception as e:
    print("‚ùå Error al verificar el estado de la tabla:", e)
    #return

# COMPROBAR Y CAMBIAR FECHAS EN EL REGISTRO DE CONTROL Y ENTREGA EN CONEXIONES RPA

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from services.login_service import login
from services.autorizacion_service import check_authorization
from services.control_entregas_service import manage_delivery_control
from services.process_autorization import process_authorization
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from selenium.webdriver.remote.remote_connection import LOGGER
import logging

# Configurar nivel de logging
LOGGER.setLevel(logging.WARNING)
logging.basicConfig(level=logging.WARNING)
logging.getLogger('urllib3').setLevel(logging.WARNING)

In [ ]:
numeros_autorizacion = [
    #"29766906",
    #"29799161"
    #"29766636",
    #"29766521",
    #"29809259",
    #"29809115",
    #"29808833",
    #"29808738"
]

In [2]:
# En la parte donde inicias el navegador, agregar opciones para suprimir logs:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--log-level=3')
options.add_argument('--disable-logging')

# Iniciar navegador con las opciones
driver = webdriver.Chrome(options=options)
# Iniciar sesi√≥n una sola vez al principio
#driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15)
login(driver, wait)

In [36]:
authorization_number = [
    #"29102191", #Video de paula
    #"29766906",
    #"29766636",
    "29766521",
    #"29820829", #ANULADA, 
    #"29799161", #ANULADA,
    #"29809259",
    #"29809115",
    #"29808833",
    #"29808738"

]

time.sleep(0.5)

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//h3[a[text()='Autorizaciones']]"))).click().click()
    time.sleep(1)
except:
    pass

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//span[text()='Autorizaciones']/parent::a"))).click()
    print("‚úÖ Entraste al m√≥dulo de 'Autorizaciones'")
except Exception as e:
    print("‚ùå No se pudo acceder al enlace de 'Autorizaciones':", e)
    #return

#try:
# Solo colapsar si est√° colapsado
#    panel = wait.until(EC.element_to_be_clickable((By.XPATH, "//h3[a[text()='Autorizaciones']]")))
#    if "collapsed" in panel.get_attribute("class"):  # o cualquier validaci√≥n del estado
#        panel.click()
#        time.sleep(1)
#except Exception as e:
#    print("‚ö†Ô∏è No se pudo verificar o expandir 'Autorizaciones':", e)

#try:
#    wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Autorizaciones']/parent::a"))).click()
#    print("‚úÖ Entraste al m√≥dulo de 'Autorizaciones'")
#except Exception as e:
#    print("‚ùå No se pudo acceder al enlace de 'Autorizaciones':", e)
#    return


try:
    input_search = wait.until(EC.presence_of_element_located(
        (By.ID, "frmAutorizaciones:tablaRegistros:j_idt80")))
    input_search.clear()
    input_search.send_keys(authorization_number)
except Exception as e:
    print("‚ùå No se pudo ingresar el n√∫mero de autorizaci√≥n:", e)
    #return

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//span[contains(@class, 'ui-icon-refresh')]/ancestor::button"))).click()
except Exception as e:
    print("‚ùå No se pudo hacer clic en Refrescar:", e)
    #return
    
time.sleep(0.5)

try:
    # Esperar a que al menos una row con la columna "Estado" est√© visible
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//td[span[@class="ui-column-title" and text()="Estado"]]'))
    )
 
    # Obtener la primera celda con columna "Estado"
    cell = driver.find_element(By.XPATH, '(//td[span[@class="ui-column-title" and text()="Estado"]])[1]')
    full_text = cell.text.strip()
 
    # Remover el encabezado si viene incluido
    if full_text.startswith("Estado"):
        text=full_text.replace("Estado", "").strip()
    else:
        text=full_text
    print(f"Estado {text}")

    estado = text
    if estado != "Autorizada":
            print("‚ö†Ô∏è El estado no es 'Autorizada'. Se detiene el proceso.")
            #return None, estado
            #return
    else:
        print("‚úÖ El estado es 'Autorizada'. Se proceder√° con el clic en 'Ver'.")
        pass
            #except Exception as e:
            #    print("‚ùå Error al verificar el estado de la tabla:", e)
            #return
except Exception as e:
    print(f"‚ùå No se pudo obtener el estado de la primera row: {e}")
    #return False
    
try:
    # Esperar a que se cierre completamente el di√°logo anterior
    time.sleep(2)
        
    # Diccionario para almacenar los results
    results = {
        'success': False,
        'data': None,
        'error': None,
        'filepath': None
    }

    # 1. Hacer clic en "Control y Registro de Entregas"
    try:
        control_button = wait.until(EC.element_to_be_clickable((
            By.XPATH, "//button[@title='Control y registro de entregas' and contains(@class, 'ui-button')]"
        )))
        control_button.click()
        print("‚úÖ Se hizo clic en 'Control y registro de entregas' correctamente.")
    except Exception as e:
        error_msg = f"No se pudo hacer clic en el bot√≥n de control: {str(e)}"
        print(f"‚ùå {error_msg}")
        results['error'] = error_msg
        #return results

except Exception as e:
        error_msg = f"Error inesperado en gestionar_control_entregas: {str(e)}"
        print(f"‚ùå {error_msg}")

try:
    # Esperar a que al menos una fila con la columna "Tipo Entrega" est√© visible
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//td[span[@class="ui-column-title" and text()="Tipo Entrega"]]'))
    )
 
    # Obtener la primera celda con columna "Tipo Entrega"
    cell = driver.find_element(By.XPATH, '(//td[span[@class="ui-column-title" and text()="Tipo Entrega"]])[1]')
    full_text = cell.text.strip()
 
    # Remover el encabezado si viene incluido
    if full_text.startswith("Tipo Entrega"):
        text=full_text.replace("Tipo Entrega", "").strip()
    else:
        text=full_text
        print(f"Tipo Entrega {text}")

    estado = text
    if estado != "Sin Entrega":
            print(" ‚úÖ El estado Tipo Entrega 'Autorizada'. Se detiene el proceso.")
            #return None, estado
            #return
    elif estado == "Sin Entrega":
        print("‚ö†Ô∏è El Tipo Entrega es 'Sin Entrega'. Se proceder√° con el clic en 'Entrega'.")
        pass
            #except Exception as e:
            #    print("‚ùå Error al verificar el estado de la tabla:", e)
            #return
except Exception as e:
    print(f"‚ùå No se pudo obtener el estado de la primera fila: {e}")
    #return False


try:
    # Esperar a que aparezca la tabla con los botones de entrega (ajusta el tiempo si es necesario)
    entrega_button = wait.until(EC.element_to_be_clickable((
        By.XPATH, "//button[@title='Entregar' and contains(@class, 'ui-button-icon-only')]"
    )))
    entrega_button.click()
    print("‚úÖ Se hizo clic en el bot√≥n 'Entregar' correctamente.")
except Exception as e:
    print(f"‚ùå No se pudo hacer clic en el bot√≥n 'Entregar': {e}")



from selenium.webdriver.common.keys import Keys

fecha = "12/06/2025"

try:
    # Esperar a que est√© presente el campo de fecha
    input_fecha = wait.until(EC.presence_of_element_located((
        By.ID, "frmEntrega:fechaEntregaGestionar_input"
    )))

    # Limpiar el campo con .clear() y/o teclas especiales
    input_fecha.click()
    input_fecha.send_keys(Keys.CONTROL + "a")  # Seleccionar todo
    input_fecha.send_keys(Keys.DELETE)         # Borrar contenido

    # Ingresar la nueva fecha (en el formato que el sistema espera, por ejemplo: dd/mm/yyyy)
    fecha_deseada = fecha
    input_fecha.send_keys(fecha_deseada)
    print(f"‚úÖ Fecha ingresada correctamente: {fecha_deseada}")

except Exception as e:
    print(f"‚ùå No se pudo ingresar la fecha en el campo: {e}")

try:
    # Esperar a que el bot√≥n de cerrar (√≠cono X) est√© presente y clickeable
    close_button = wait.until(EC.element_to_be_clickable((
        By.XPATH, "//div[contains(@class, 'ui-dialog-titlebar') and .//span[text()='Entregar']]//a[contains(@class, 'ui-dialog-titlebar-close')]"
    )))
    close_button.click()
    print("‚úÖ Se cerr√≥ la ventana de 'Entregar' correctamente.")
except Exception as e:
    print(f"‚ùå No se pudo cerrar la ventana de 'Entregar' porque no aparecio o el tipo de entregada es el adecuado: {e}")


# 3. Cerrar el di√°logo de Control y Registro de Entregas (versi√≥n mejorada)
try:
    # Esperar a que el di√°logo est√© completamente cargado
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.ui-dialog[aria-labelledby='j_idt281_title']")))
            
    # Opci√≥n 1: Intentar cerrar con el bot√≥n X (versi√≥n m√°s espec√≠fica)
    try:
        close_button = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, "div.ui-dialog[aria-labelledby='j_idt281_title'] a.ui-dialog-titlebar-close")
        ))
        close_button.click()
        print("‚úÖ Di√°logo de control cerrado con el bot√≥n X (selector espec√≠fico)")
    except Exception as e:
        print(f"‚ö†Ô∏è No se pudo cerrar con bot√≥n X (intentando alternativa): {str(e)}")
                
    # Opci√≥n 2: Intentar con el bot√≥n Salir si existe
    #try:
    #    exit_button = wait.until(EC.element_to_be_clickable(
    #        (By.XPATH, "//div[@id='j_idt281']//button[contains(text(), 'Salir')]")
    #    ))
    #    exit_button.click()
    #    print("‚úÖ Di√°logo de control cerrado con el bot√≥n Salir")
    #except Exception as e:
    #    print(f"‚ö†Ô∏è No se pudo cerrar con bot√≥n Salir (intentando JavaScript): {str(e)}")
                    
    # Opci√≥n 3: Forzar cierre con JavaScript
    try:
        driver.execute_script("""
        var dialogs = document.querySelectorAll('div.ui-dialog');
        dialogs.forEach(function(dialog) {
        var title = dialog.querySelector('.ui-dialog-title');
        if(title && title.textContent.includes('Control y registro de entregas')) {
        var closeBtn = dialog.querySelector('a.ui-dialog-titlebar-close');
        if(closeBtn) closeBtn.click();
        }
        });
        """)
        print("‚úÖ Di√°logo de control cerrado mediante JavaScript")
    except Exception as js_e:
        print(f"‚ö†Ô∏è Fallo al cerrar con JavaScript: {str(js_e)}")
                        
except Exception as e:
    print(f"‚ùå Error grave al intentar cerrar el di√°logo: {str(e)}")


‚úÖ Entraste al m√≥dulo de 'Autorizaciones'
Estado Autorizada
‚úÖ El estado es 'Autorizada'. Se proceder√° con el clic en 'Ver'.
‚úÖ Se hizo clic en 'Control y registro de entregas' correctamente.
Tipo Entrega Total
 ‚úÖ El estado Tipo Entrega 'Autorizada'. Se detiene el proceso.
‚ùå No se pudo hacer clic en el bot√≥n 'Entregar': Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff6d642fe75+79173]
	GetHandleVerifier [0x0x7ff6d642fed0+79264]
	(No symbol) [0x0x7ff6d61e9e5a]
	(No symbol) [0x0x7ff6d6240586]
	(No symbol) [0x0x7ff6d624083c]
	(No symbol) [0x0x7ff6d6294247]
	(No symbol) [0x0x7ff6d62689af]
	(No symbol) [0x0x7ff6d629100d]
	(No symbol) [0x0x7ff6d6268743]
	(No symbol) [0x0x7ff6d62314c1]
	(No symbol) [0x0x7ff6d6232253]
	GetHandleVerifier [0x0x7ff6d66fa2ad+3004797]
	GetHandleVerifier [0x0x7ff6d66f46fd+2981325]
	GetHandleVerifier [0x0x7ff6d6713350+3107360]
	GetHandleVerifier [0x0x7ff6d644a9fe+188622]
	GetHandleVerifier [0x0x7ff6d645228f+219487]
	GetHandleVerifier [0x0x7ff6d6438dc4+11586

In [ ]:
#OPCIONAL 
try:
    # Esperar a que aparezca la tabla con los botones de entrega (ajusta el tiempo si es necesario)
    entrega_button = wait.until(EC.element_to_be_clickable((
        By.XPATH, "//button[@title='Ver Entregas' and contains(@class, 'ui-button-icon-only')]"
    )))
    entrega_button.click()
    print("‚úÖ Se hizo clic en el bot√≥n 'Ver Entregas' correctamente.")
except Exception as e:
    print(f"‚ùå No se pudo hacer clic en el bot√≥n 'Entregar': {e}")